In [2]:
#import PIL
import cv2
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:
#PIL Import
#images = [PIL.Image.open('pics/'+x) for x in os.listdir('pics/')]
images = [compressable_image(cv2.imread('pics/'+x)) for x in os.listdir('pics/')]

In [11]:
class compressable_image:
    def __init__(self, image):
        
        self.M = len(image)
        self.N = len(image[0])
        self.unrolled_image = self.unroll_image(image)
        self.image = []
        
    def unroll_image(self, original_image):
        new_image = []
        for i, row in enumerate(original_image):
            for j, pixel in enumerate(row):
                new_image.append(pixel)
        return new_image
    
    def k_means_compression(self, K=16, iteration_limit = 10):
        
        # ||x-mu||**2
        euclidean_distance = lambda x: np.inner(x,x)
        
        centroids = np.random.randint(255, size=(K, 3))
        last = np.zeros(self.M*self.N)
        assignments = np.zeros(self.M*self.N)
        changed = True
        iteration = 0
        while iteration < iteration_limit:
            for i, pixel in enumerate(self.unrolled_image):  
                assigned_k = -1
                closest = float('inf')
                for k, centroid in enumerate(centroids):
                    dist = euclidean_distance(pixel-centroids[k])
                    if dist < closest:
                        assigned_k, closest = k, dist
                assignments[i] = assigned_k
        
            if np.array_equal(last, assignments):
                break
            else:
                last = np.copy(assignments)
                assignments = np.zeros(self.M*self.N)
                
            for i, centroid in enumerate(centroids):
                points = [self.unrolled_image[x] for x in range(len(last)) if last[x] == i]
                if len(points) != 0:
                    new_centroid = np.sum(points, axis = 0)/len(points)
                    centroids[i] = new_centroid
            iteration += 1
            print(iteration)
        
        for pixel in range(len(self.unrolled_image)):
            self.unrolled_image[pixel] = centroids[int(last[pixel]),:]

        self.reroll_image()
        
    def reroll_image(self):
        
        self.image = []
        ind = 0
        for i in range(self.M):
            self.image.append([])
            for j in range(self.N):
                self.image[-1].append([])
                for d in range(3):
                    self.image[-1][-1].append(self.unrolled_image[ind][d])
                ind += 1
        self.image = np.asarray(self.image).astype(np.uint8)
        

In [12]:
images = [compressable_image(cv2.imread('pics/'+x)) for x in os.listdir('pics/')]

In [13]:
images[0].k_means_compression()

KeyboardInterrupt: 

In [ ]:
centroids = [np.random.randint(255, size=(1, 3)) for _ in range(5)]

In [ ]:
centroids

In [ ]:
np.sum(centroids, axis = 0)/len(centroids)

In [4]:
testimg = cv2.imread('pics/test.png')

In [5]:
test = compressable_image(testimg)

In [6]:
test.k_means_compression()

In [86]:
mat = []
for i in range(len(testimg)):
    mat.append([])
    for j in range(len(testimg[i])):
        mat[-1].append([])
        for k in range(3):
            mat[-1][-1].append(testimg[i][j][k])
mat = np.asarray(mat)
test.image.shape

(128, 128, 3)

In [ ]:
#test.image
#img = cv2.imwrite('test_img.jpg', test.image)
#img = cv2.imdecode(mat,-1)
# The important part - Correct BGR to RGB channel
img = cv2.cvtColor(images[0].image, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
a =np.array([])

In [ ]:
np.append(a,5)
a

In [ ]:
a = np.append(a,5)

In [ ]:
a

In [ ]:
a = np.append(a,np.array)

In [ ]:
a